In [ ]:
from src import *
import ffmpegio
from data import datasets
gpu = 0

In [ ]:
def load_experiment(results_dir, model):
    param = torch.load(os.path.join(results_dir, "param.pth"))
    spec = importlib.util.spec_from_file_location("module.name", os.path.abspath(os.path.join(results_dir, os.path.basename(param.experiment.script_file_path))))
    experiment_script = importlib.util.module_from_spec(spec)
    sys.modules["module.name"] = experiment_script
    spec.loader.exec_module(experiment_script)
    exp = experiment_script.load_experiment(results_dir, model, gpu=gpu)
    return exp

In [ ]:
dataset_nr = 10
results_dir = "results/10/FMLP/validation/900/s_t 1.0 spatial_coordinate_scale 30.0/"
model = "training/ser_highscore.pth"
exp = load_experiment(results_dir, model)

In [ ]:
def render_mp4(imgs, export_dir, frame_times):
    create_dir(export_dir)

    imgs_rgb = imgs.copy()
    img_max_val = datasets[dataset_nr]["brightness"]

    imgs_rgb /= img_max_val
    imgs_rgb[imgs_rgb > 1.] = 1. # clip maximum
    imgs_rgb[imgs_rgb < 0] = 0. # clip minimum (should not be necessary for absolute valued images)
    imgs_rgb *= 255
    imgs_rgb = np.uint8(imgs_rgb)

    imgs_rgb = np.expand_dims(imgs_rgb, axis=3)
    imgs_rgb = imgs_rgb.repeat(3, axis=3)

    raw_video_file = os.path.join(export_dir, '30fps_cfr.mp4')

    ffmpegio.video.write(raw_video_file, 30, imgs_rgb, overwrite=True)

    # write timecode file

    timecode = """# timecode format v2

    """
    for i, t_k in enumerate(frame_times):
        if i >= len(imgs): break
        timecode += "{}\n".format(t_k * 1000)

    timecodes_file = os.path.join(export_dir, 'timecodes.txt')
    with open(timecodes_file, 'w') as f:
        f.write(timecode)

    timecoded_vfr_file = os.path.join(export_dir, 'timecoded_vfr.mp4')
    timecoded_cfr_file = os.path.join(export_dir, 'timecoded_cfr.mp4')


    os.system("mp4fpsmod -o \"{}\" -t \"{}\" \"{}\"".format(timecoded_vfr_file, timecodes_file, raw_video_file))
    os.system("ffmpeg -y -i \"{}\" \"{}\"".format(timecoded_vfr_file, timecoded_cfr_file))


if dataset_nr == 10 or dataset_nr == 15:
    imgs = np.stack([np.rot90(exp.model.evaluate_npy(sample)[26:-26,65:-65]) for sample in exp.dataset])
else:
    imgs = np.stack([np.rot90(exp.model.evaluate_npy(sample)[47:-47,120:-120]) for sample in exp.dataset])

export_dir = "media/lowres_highsnr/FMLP/900/"
render_mp4(imgs, export_dir, exp.param.data.frame_times)


In [ ]:
all_images = []

images = [exp.model.evaluate_npy(exp.dataset[k]) for k in [105, 115, 125, 135]]
if dataset_nr == 10 or dataset_nr == 15: # low-res
    images = [np.rot90(img[26:-26,65:-65]) for img in images]
else: # high-res
    images = [np.rot90(img[47:-47,120:-120]) for img in images]
all_images.append(images)

import glob, tikzplotlib

fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(9, 9))
brightness = datasets[dataset_nr]["brightness"]
for i, img in enumerate(images):
    axes[i].imshow(img, cmap="gray", vmax=brightness, vmin=0)

create_dir(os.path.join(results_dir, "figures"))
tex_file_name = os.path.join(results_dir, "figures/fmlp.tex")
for file in glob.glob(os.path.splitext(tex_file_name)[0] + "-*"):
    os.remove(file)
tikzplotlib.save(tex_file_name, standalone=False)